<a href="https://colab.research.google.com/github/Brunofell/Arquivos_JAVA/blob/main/TDE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TDE 3
# NOME: BRUNO FELICIANO, LEANDRO GABRIEL, THIAGO LUZ

In [ ]:
!pip install pyspark==3.3.1

In [3]:
# abre sessão no spark
import pyspark
conf = pyspark.SparkConf()
conf.setMaster('local[*]')
sc = pyspark.SparkContext.getOrCreate(conf)

In [4]:
# Carregando um arquivo no RDD
rdd = sc.textFile('/content/transactions_amostra.csv')

In [6]:
# 1- The number of transactions involving Brazil;
rdd.filter(lambda x: x.split(';')[0] in "Brazil").count()

6441

In [ ]:
# 2- The number of transactions per flow type and year;

print("Número de transações por tipo de fluxo: ")
rddExport = rdd.filter(lambda linha: linha.split(';')[4]=="Export").count()
print("Export: ", rddExport)
rddReExport = rdd.filter(lambda linha: linha.split(';')[4]=="Re-Export").count()
print("Re-Export: ", rddReExport)
rddImport = rdd.filter(lambda linha: linha.split(';')[4]=="Import").count()
print("Import: ", rddImport)

print(" ")
print("Número de transações por ano: ")
# mapeando cada linha para uma tupla com ano e valor
rddTransactionYear = rdd.map(lambda coluna: (coluna.split(';')[1], 1))\
.reduceByKey(lambda x, y: x+y) #reduzindo pra contar o número de transações por ano

# aqui estamos coletando os resultados e imprimindo
result = rddTransactionYear.collect()

for c in result:
  print(f"ano: {c[0]}, Número de transações: {c[1]} ")


In [ ]:
# 3- The average of commodity values per year;

# Mapeie cada linha (ano, valor da commodity) usando linha 1 e 5
rddAnoCommodity = rdd.map(lambda coluna: (coluna.split(';')[1], coluna.split(';')[5]))
# Filtre os valores inválidos (que não podem ser convertidos para float)
rddValidacao = rddAnoCommodity.filter(lambda item: item[1].replace('.', '', 1).isdigit())
# Converta os valores para float
rddFloat = rddValidacao.map(lambda valor: (valor[0], float(valor[1])))
# Agrupe as tuplas pelo ano
rddPorAno = rddFloat.groupByKey();
# Calcule a média dos valores das commodities para cada ano
rddMedia = rddPorAno.map(lambda x: (x[0], sum(x[1])/len(x[1])))
# Coletar os resultados e imprimir
result = rddMedia.collect()
for c in result:
  print(f"Ano: {c[0]}, Média dos Valores das Commodities: {c[1]}")

In [ ]:
# 4- The average price of commodities per unit type, year, and category in the export flow in Brazil;
# Mapeie as linhas e filtre para restringir a análise apenas para exportações do Brasil
def parse_line(line):
    fields = line.split(';')
    if len(fields) >= 10 and "Brazil" in fields[0] and fields[4] == "Export":
        try:
            price = float(fields[5])
            return ((fields[1], fields[9]), (price, 1))
        except ValueError:
            return None
    else:
        return None

# Filtrar apenas as exportações do Brasil e calcular a soma de preços e o total de ocorrências
filtered_data = rdd.map(parse_line).filter(lambda x: x is not None)
reduced_data = filtered_data.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

# Calcule a média de preços por ano, categoria e tipo de unidade
average_data = reduced_data.map(lambda x: (x[0], x[1][0] / x[1][1]))

# Classifique os resultados por ano, categoria e tipo de unidade
sorted_data = average_data.sortBy(lambda x: (x[0][0], x[0][1]))

# Salvar o resultado no diretório de saída
sorted_data.collect()



In [ ]:
# 5- The maximum, minimum, and mean transaction price per unit type and year;

file_path = "transactions_amostra.csv"

data = sc.textFile(file_path)

unit_year_prices = data.map(lambda line: ((line.split(";")[1], line.split(";")[7]), line.split(";")[5]))
unit_year_prices = unit_year_prices.filter(lambda item: item[1].replace('.', '', 1).isdigit())  # Filtra apenas valores numéricos
unit_year_prices = unit_year_prices.map(lambda item: (item[0], float(item[1])))

# Calcule a média de preços por tipo de unidade e ano
unit_year_prices_mean = unit_year_prices.combineByKey(
    lambda value: (value, 1),
    lambda x, value: (x[0] + value, x[1] + 1),
    lambda x, y: (x[0] + y[0], x[1] + y[1])
)

# Calcule o preço máximo e mínimo por tipo de unidade e ano
unit_year_prices_max = unit_year_prices.reduceByKey(lambda x, y: max(x, y))
unit_year_prices_min = unit_year_prices.reduceByKey(lambda x, y: min(x, y))

# Formate os números para o formato xxx,xx
formatted_unit_year_prices_mean = unit_year_prices_mean.map(lambda item: (item[0], "{:.2f}".format(item[1][0] / item[1][1]).replace(".", ",")))
formatted_unit_year_prices_max = unit_year_prices_max.map(lambda item: (item[0], "{:.2f}".format(item[1]).replace(".", ",")))
formatted_unit_year_prices_min = unit_year_prices_min.map(lambda item: (item[0], "{:.2f}".format(item[1]).replace(".", ",")))

# Imprima os preços mínimo, médio e máximo por tipo de unidade e ano com separadores
print("Preço mínimo por tipo de unidade e ano:")
for item in formatted_unit_year_prices_min.collect():
    print(item[0], item[1])

print("----" * 10)  # Linha separadora

print("Preço médio por tipo de unidade e ano:")
for item in formatted_unit_year_prices_mean.collect():
    print(item[0], item[1])

print("----" * 10)  # Linha separadora

print("Preço máximo por tipo de unidade e ano:")
for item in formatted_unit_year_prices_max.collect():
    print(item[0], item[1])

In [ ]:
# 6- The country with the largest average commodity price in the Export flow;

# O resultado que estava saindo era a União Europeia, ams como ela é um bloco economico de 27 países
# colocamos pra mostrar o segndo maior da lista, que dá o USA.

# Mapeie os dados para (país, preço) e filtre os valores inválidos
rddPaisPreco = rdd.map(lambda coluna: (coluna.split(';')[0], coluna.split(';')[5].replace(',', '')))
rddPaisPreco = rddPaisPreco.filter(lambda x: x[0] != 'country' and x[1].replace('.', '', 1).isdigit())

# Converta para float
rddPaisPreco = rddPaisPreco.map(lambda x: (x[0], float(x[1])))

# Agrupe os dados por país
rddPorPais = rddPaisPreco.groupByKey()

# Calcule a média dos preços das commodities para cada país
rddMediaPorPais = rddPorPais.map(lambda x: (x[0], sum(x[1]) / len(list(x[1]))))

# Encontre o país com a maior média de preços de commodities
maior_media = rddMediaPorPais.max(lambda x: x[1])

# Encontre o segundo país com a maior média de preços de commodities, excluindo o país com a maior média
segunda_maior_media = rddMediaPorPais.filter(lambda x: x[0] != maior_media[0]).top(1, key=lambda x: x[1])


# Imprima o resultado

# print(f"O país com o maior preço médio de commodities no fluxo de exportação é: {maior_media[0]} com preço médio de {maior_media[1]}")

print(f"O país com o maior preço médio de commodities no fluxo de exportação é: {segunda_maior_media[0][0]} com preço médio de {segunda_maior_media[0][1]}")